In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [17]:
sq_eu = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/maps/QGIS/square_europe_updated.csv')
storm_dates = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/tracks/storms_dates_work.csv',
                          delimiter = ';')
name_storm = pd.read_csv('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/tracks/C3S_StormTracks_era5_19792021_0100_v3.csv',
                         delimiter=';')

landfall_eu = sq_eu.drop_duplicates(subset=['layer'], keep='first')
landfall_eu = landfall_eu.drop(columns=['lon_east', 'lon_west', 'lat_north', 'lat_south', 'path','center_lon','center_lat'])

name_storm_index = name_storm[name_storm['Time&Longitude&Latitude'].str.startswith('TRACK_ID')]
name_storm = name_storm[name_storm['Time&Longitude&Latitude'].str.startswith('TRACK_ID')]

In [30]:
import re

# Input string
line = name_storm[name_storm['Time&Longitude&Latitude'].str.startswith('TRACK_ID')]

# Use regex to find the number after TRACK_ID and the text after 0
all_storms = len(name_storm)
for storm in range(0,all_storms):
    match = re.search(r'TRACK_ID (\d+).*?ENSMBLE_MEMB 0 (.+)', line['Time&Longitude&Latitude'].loc[storm])

if match:
    track_id_value = match.group(1)
    after_zero = match.group(2)

    print("TRACK_ID value:", track_id_value)
    print("Text after 0:", after_zero)

KeyError: 1

In [33]:
# in the column 'layer' rename each fid_reproj_tc_irad_i_interp by storm_i

landfall_eu = landfall_eu.rename(columns={'layer': 'storm_index'})
landfall_eu['storm_index'] = landfall_eu['storm_index'].str.replace('fid_reproj_tc_irad_', '')
landfall_eu['storm_index'] = landfall_eu['storm_index'].str.replace('_interp', '')

# clean the storm_dates dataframe

storm_eu_dates = storm_dates.drop(columns=['end_date', '3_hour_steps', '1_hour_steps'])
landfall_eu['storm_index'] = pd.to_numeric(landfall_eu['storm_index'], errors='coerce')

# keep only the name of the storms

name_storm = name_storm['Time&Longitude&Latitude'].str.split(' 0 ', n=1).str[1].str.strip()
name_storm = name_storm.apply(lambda x: x.replace('NAME', '', 1).strip() if x.startswith('NAME') else x)
name_storm.reset_index(drop=True, inplace=True)
name_storm = pd.DataFrame(name_storm)

In [4]:
'''test = storm_eu_dates['start_date'][0+1]
hours_to_add = landfall_eu.loc[landfall_eu['storm_index'] == 1]
hours_to_add = int(hours_to_add['fid'].values[0])
date_time_obj = datetime.strptime(test, '%Y-%m-%dT%H:%M:%S')
date_time_obj = date_time_obj + timedelta(hours=hours_to_add)

# convert back to string

date_time_obj.strftime('%Y-%m-%dT%H:%M:%S')'''

"test = storm_eu_dates['start_date'][0+1]\nhours_to_add = landfall_eu.loc[landfall_eu['storm_index'] == 1]\nhours_to_add = int(hours_to_add['fid'].values[0])\ndate_time_obj = datetime.strptime(test, '%Y-%m-%dT%H:%M:%S')\ndate_time_obj = date_time_obj + timedelta(hours=hours_to_add)\n\n# convert back to string\n\ndate_time_obj.strftime('%Y-%m-%dT%H:%M:%S')"

In [120]:
storm_eu_arrivals = []
steps_added = []

for i in range(len(storm_eu_dates)):
    try:
        start = storm_eu_dates['start_date'][i]
        hours_to_add = landfall_eu.loc[landfall_eu['storm_index'] == i+1]
        hours_to_add = int(hours_to_add['fid'].values[0])
        start_reformat = datetime.strptime(start, '%Y-%m-%dT%H:%M:%S')
        start_reformat = start_reformat + timedelta(hours=hours_to_add)
        storm_eu_arrivals.append(start_reformat.strftime('%Y-%m-%dT%H:%M:%S'))
        steps_added.append(hours_to_add)
    except:
        print(f'storm {i} not found')
        storm_eu_arrivals.append(-1)
        steps_added.append(-1)

storm 90 not found


In [6]:
# add the column 'arrival_date' to storm_dates

storm_dates['eu_landfall_date'] = storm_eu_arrivals
storm_dates['nb_steps_before_eu_landfall_1_hour'] = steps_added
storm_dates['storm_name'] = name_storm

In [137]:
matched_data = []
df_storm_eu_arrivals= []
df_steps_added = []

df_storm_eu_arrivals = pd.DataFrame(storm_eu_arrivals)
df_steps_added = pd.DataFrame(steps_added)

# Loop through storm_dates and find matches
for idx in range(0,96+1):
    if idx in name_storm.index:
        matched_storm_name = name_storm['Time&Longitude&Latitude'].loc[idx]
        matched_storm_eu_arrivals = df_storm_eu_arrivals.loc[idx][0]
        matched_steps_added = df_steps_added.loc[idx][0]
        matched_data.append([idx+1, matched_storm_eu_arrivals,matched_steps_added, matched_storm_name])

# Convert matched data to a new DataFrame
matched_df = pd.DataFrame(matched_data, columns=['storm_index_match','storm_eu_arrivals', 'steps_added','storm_name'])
print(matched_df)

    storm_index_match    storm_eu_arrivals  steps_added  \
0                   1  1990-01-23T17:00:00           62   
1                   2  1990-01-25T05:00:00           50   
2                   3  1990-02-01T13:00:00           91   
3                   4  1990-02-02T16:00:00           40   
4                   5  1990-02-20T18:00:00           54   
..                ...                  ...          ...   
91                 92  2020-02-09T07:00:00           10   
92                 93  2020-02-15T15:00:00           54   
93                 94  2020-10-01T21:00:00           21   
94                 95  2020-10-31T04:00:00           46   
95                 96  2021-03-23T19:00:00           49   

                     storm_name  
0         C3S_STORM_TRACKS_ERA5  
1                         DARIA  
2         C3S_STORM_TRACKS_ERA5  
3         C3S_STORM_TRACKS_ERA5  
4                        VIVIAN  
..                          ...  
91                       CIARA2  
92                 

In [142]:
final_df = pd.concat([storm_dates, matched_df], axis=1)
final_df = final_df.drop(columns=['Unnamed: 0.1'])

In [143]:
# save the new storm_dates

final_df.to_csv('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/pre_processing/tracks/storms_dates_steps_index_landfall_corrected.csv')